<a href="https://colab.research.google.com/github/wenchun0731/city-hunter/blob/main/plates_identify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pytesseract
import os
import sys


class plates_OCR:
  def __init__(self,image):
    self.image=image

  def main(self):
    photo_resize = self.photo_resize(self.image, 320, 136)
    self.adj_img = self.adjusted(photo_resize, -1.5, 15)
    gray_img = cv2.cvtColor(self.adj_img, cv2.COLOR_BGR2GRAY)
    Sobelx = self.Sobel(gray_img)
    blur_img = self.Gussian(Sobelx)
    binary = self.threshold(blur_img)
    min_x, max_x = self.contours(binary)

    if min_x == float('inf') or max_x == 0:
      print("無法識別車牌範圍")
      return

    cropped_img = self.cropped(binary, min_x, 0, binary.shape[0], max_x)
    ero_img = self.erope(cropped_img)
    dilate_img = self.dilate(ero_img)
    bottom_left, top_left, bottom_right, top_right = self.findcontour(dilate_img)

    if np.all(bottom_left == 0):
      print("未找到輪廓")
      return

    self.adj_img = self.cropped(self.adj_img, min_x, 0, self.adj_img.shape[0], max_x)
    self.affine(self.adj_img, bottom_left, top_left, bottom_right, top_right)
    self.bg('output.jpg')
    text = self.tesseracttt('output.jpg')

    return text


  def photo_resize(self,img,target_width,target_height):
    # 調整圖片大小
    re_img=cv2.resize(img, (target_width, target_height))
    return re_img
  #調整亮度與對比度
  def adjusted(self,img,x,y):
    alpha = x
    beta = y

    adjusted_image = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    return adjusted_image

  def Sobel(self,img):
    Sobelx=cv2.Sobel(img,cv2.CV_32F,1,0)
    Sobelx=cv2.convertScaleAbs(Sobelx)
    return Sobelx

  def Gussian(self,img):
    kernel_size = (5, 5)  # 高斯和大小 通常奇數
    sigma_x = 0  # X方向上的標準差，0 表示根據核大小自動計算
    # 高斯濾波器
    blurred_img = cv2.GaussianBlur(img, kernel_size, sigma_x)
    return blurred_img
  def threshold(self,img):
    thr, binary = cv2.threshold(img,127, 255, cv2.THRESH_BINARY)
    return binary
  def contours(self,img):
    contours1 = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #尋找輪廓
    contours = contours1[0]   #取得輪廓
    # cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
    letters=[]
    for contour in contours:
      x,y,w,h=cv2.boundingRect(contour)
      if 15<h<85: #要改的參數!!!10<h<100
        letters.append(contour)
    # cv2.drawContours(contours_img, letters, -1, (255, 0, 0), 2)
    # cv2_imshow(contours_img)
    #找出最小x跟最大x
    min_x =float('inf')#正無窮
    max_x =0
    for letter in letters:
      for point in letter:
        x=point[0][0]
        min_x=min(min_x,x)
        max_x=max(max_x,x)
    return min_x,max_x
  #image[y:h,x:w]
  def cropped(self,img,x,y,h,w):
    cropped_img=img[y:h,x:w]

    return cropped_img

  def erope(self,img):
    kernel=np.ones((3, 3),np.uint8)
    ero=cv2.erode(img,kernel,iterations=1)
    return ero

  def dilate(self,img):
    kernel=np.ones((2, 5),np.uint8)
    dilation=cv2.dilate(img,kernel,iterations=8)
    return dilation


  def findcontour(self, img):
      contours1 = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #尋找輪廓
      contours = contours1[0]   #取得輪廓
      points = None
      min_contour_length = 2500

      for contour in contours:
          area = cv2.contourArea(contour)

          if area >= min_contour_length:
              rect = cv2.minAreaRect(contour)
              angle = rect[2]
              points = cv2.boxPoints(rect)
              points = np.int0(points)
              break

      if points is not None:
          sorted_points = sorted(points, key=lambda x: x[0])
          bottom_left = sorted_points[0]
          top_left = sorted_points[1]
          bottom_right = sorted_points[2]
          top_right = sorted_points[3]

          return bottom_left, top_left, bottom_right, top_right
      else:
          return np.array([0, 0]), np.array([0, 0]), np.array([0, 0]), np.array([0, 0])

  def affine(self,img,bottom_left,top_left,bottom_right,top_right):
    p1 = np.float32([(top_left),(top_right),(bottom_right)])
    p2 = np.float32([[top_left[0],top_right[1]],(top_right),(bottom_right)])
    M = cv2.getAffineTransform(p1, p2)

    h,w=self.adj_img.shape[:2]

    output = cv2.warpAffine(self.adj_img, M,(w,h))

    x=0
    y=min(top_right[1],bottom_right[1])
    h=abs(bottom_right[1]-top_right[1])

    output=output[y-5:y+h,x:x+w]
    # output=adjusted(output,-1.5,15)
    # output=adjusted(output,-1.3,12)
    # output=adjusted(output,-1.5,15)
    output=self.adjusted(output,-1.2,13)
    if output is not None:
      # output = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)
      # thr, output = cv2.threshold(output,127, 255, cv2.THRESH_BINARY)
      cv2.imwrite('output.jpg',output)
    else:
      print('None')
      pass

  def bg(self,img):
    image=cv2.imread(img)
    height,width=image.shape[:2]
    new_height=height+36
    new_width=width+36
    x_offset=18
    y_offset=18
    #創建一個白色背景圖像
    background = np.ones((new_height, new_width, 3), dtype=np.uint8) * 255
    background[y_offset:y_offset+height,x_offset:x_offset+width]=image
    cv2.imwrite('output.jpg',background)

  def tesseracttt(self,img):

    config = r'-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789. --oem 3 --psm 6'
    text = pytesseract.image_to_string(img,lang='eng',config=config)

    if len(text) ==1:
      return None
      print('辨識失敗')
    else:
      clean_text = text.strip()#去除兩端不可見字符
      print('辨識結果:\n{}'.format(clean_text))
      # img=cv2.imread(img)
      # cv2.putText(img, clean_text, (int(110), int(70)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
      # cv2_imshow(img)
      return text


辨識結果:
AUM0392


<ipython-input-8-7e70f2660968>:126: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  points = np.int0(points)
